In [1]:
import numpy as np
import pandas as pd
import pickle
import os

from scipy.spatial.distance import cdist
from mpl_toolkits.mplot3d.axes3d import Axes3D

from sklearn.preprocessing import StandardScaler, RobustScaler

# Кластеризация
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

from sklearn.neighbors import KNeighborsClassifier

# Понижения размерности
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse

from sklearn.ensemble import GradientBoostingRegressor

import warnings
warnings.filterwarnings('ignore')

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor

In [2]:
train_df=pd.read_csv('train.csv')

In [3]:
test_df=pd.read_csv('test.csv')

In [4]:
train_df.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [5]:
train_df.head()


,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,11809,27,3.0,115.027311,NaN,10.0,4,10.0,2014,0.075424,B,B,11,3097,0,NaN,0,0,B,305018.871089
1,3013,22,1.0,39.832524,23.169223,8.0,7,8.0,1966,0.118537,B,B,30,6207,1,1183.0,1,0,B,177734.553407
2,8215,1,3.0,78.342215,47.671972,10.0,2,17.0,1988,0.025609,B,B,33,5261,0,240.0,3,1,B,282078.720850
3,2352,1,1.0,40.409907,NaN,1.0,10,22.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,168106.007630
4,13866,94,2.0,64.285067,38.562517,9.0,16,16.0,1972,0.282798,B,B,33,8667,2,NaN,0,6,B,343995.102962


### Заполняем пропуски LifeSquare

In [6]:
tmp_df = train_df.copy()

In [7]:
tmp_df['ind'] = [x for x in range(len(tmp_df))]
tmp_target_name = 'LifeSquare'
tmp_df.columns.tolist()

['Id',
 'DistrictId',
 'Rooms',
 'Square',
 'LifeSquare',
 'KitchenSquare',
 'Floor',
 'HouseFloor',
 'HouseYear',
 'Ecology_1',
 'Ecology_2',
 'Ecology_3',
 'Social_1',
 'Social_2',
 'Social_3',
 'Healthcare_1',
 'Helthcare_2',
 'Shops_1',
 'Shops_2',
 'Price',
 'ind']

In [8]:
tmp_feature_name = [#'Id',
 #'DistrictId',
 'Rooms',
 'Square',
 #'LifeSquare',
 'KitchenSquare',
 'Floor',
 'HouseFloor',
 'HouseYear',
 'Ecology_1',
 #'Ecology_2',
 #'Ecology_3',
 'Social_1',
 'Social_2',
 'Social_3',
 #'Healthcare_1',
 'Helthcare_2',
 'Shops_1',
 #'Shops_2',
 'Price',
 #'ind'
]

In [9]:
tmp_df_test = tmp_df[tmp_df[tmp_target_name].isna() == True]
tmp_df_train = tmp_df[tmp_df[tmp_target_name].isna() == False]

In [10]:
X_tmp_train = tmp_df_train[tmp_feature_name]
X_tmp_test = tmp_df_test[tmp_feature_name]
y_tmp_train = tmp_df_train[tmp_target_name]

In [11]:
rf_model_tmp = RandomForestRegressor(random_state=8)
rf_model_tmp.fit(X_tmp_train, y_tmp_train)
tmp_df_test[tmp_target_name] = rf_model_tmp.predict(X_tmp_test)
df2 = pd.concat([tmp_df_train, tmp_df_test])
df2 = df2.sort_values(by=['ind'])
train_df[tmp_target_name + '_num'] = df2[tmp_target_name]
train_df
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              10000 non-null  int64  
 1   DistrictId      10000 non-null  int64  
 2   Rooms           10000 non-null  float64
 3   Square          10000 non-null  float64
 4   LifeSquare      7887 non-null   float64
 5   KitchenSquare   10000 non-null  float64
 6   Floor           10000 non-null  int64  
 7   HouseFloor      10000 non-null  float64
 8   HouseYear       10000 non-null  int64  
 9   Ecology_1       10000 non-null  float64
 10  Ecology_2       10000 non-null  object 
 11  Ecology_3       10000 non-null  object 
 12  Social_1        10000 non-null  int64  
 13  Social_2        10000 non-null  int64  
 14  Social_3        10000 non-null  int64  
 15  Healthcare_1    5202 non-null   float64
 16  Helthcare_2     10000 non-null  int64  
 17  Shops_1         10000 non-null  

### Заполняем пропуски Healthcare_1

In [12]:
tmp_df = train_df.copy()
tmp_df['ind'] = [x for x in range(len(tmp_df))]
tmp_target_name = 'Healthcare_1'
tmp_df.columns.tolist()

['Id',
 'DistrictId',
 'Rooms',
 'Square',
 'LifeSquare',
 'KitchenSquare',
 'Floor',
 'HouseFloor',
 'HouseYear',
 'Ecology_1',
 'Ecology_2',
 'Ecology_3',
 'Social_1',
 'Social_2',
 'Social_3',
 'Healthcare_1',
 'Helthcare_2',
 'Shops_1',
 'Shops_2',
 'Price',
 'LifeSquare_num',
 'ind']

In [13]:
tmp_feature_name = [#'Id',
 #'DistrictId',
 'Rooms',
 'Square',
 #'LifeSquare',
 'KitchenSquare',
 'Floor',
 'HouseFloor',
 'HouseYear',
 'Ecology_1',
 #'Ecology_2',
 #'Ecology_3',
 'Social_1',
 'Social_2',
 'Social_3',
 #'Healthcare_1',
 'Helthcare_2',
 'Shops_1',
 #'Shops_2',
 'Price',
 #'LifeSquare_num',
 #'ind'
]

In [14]:
tmp_df_test = tmp_df[tmp_df[tmp_target_name].isna() == True]
tmp_df_train = tmp_df[tmp_df[tmp_target_name].isna() == False]

In [15]:
X_tmp_train = tmp_df_train[tmp_feature_name]
X_tmp_test = tmp_df_test[tmp_feature_name]
y_tmp_train = tmp_df_train[tmp_target_name]

In [16]:
rf_model_tmp = RandomForestRegressor(random_state=8)


In [17]:
rf_model_tmp.fit(X_tmp_train, y_tmp_train)


RandomForestRegressor(random_state=8)

In [18]:
tmp_df_test[tmp_target_name] = rf_model_tmp.predict(X_tmp_test)
df2 = pd.concat([tmp_df_train, tmp_df_test])
df2 = df2.sort_values(by=['ind'])
train_df[tmp_target_name + '_num'] = df2[tmp_target_name]
train_df

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_num,Healthcare_1_num
0,11809,27,3.0,115.027311,NaN,10.0,4,10.0,2014,0.075424,...,11,3097,0,NaN,0,0,B,305018.871089,72.762383,586.89
1,3013,22,1.0,39.832524,23.169223,8.0,7,8.0,1966,0.118537,...,30,6207,1,1183.0,1,0,B,177734.553407,23.169223,1183.00
2,8215,1,3.0,78.342215,47.671972,10.0,2,17.0,1988,0.025609,...,33,5261,0,240.0,3,1,B,282078.720850,47.671972,240.00
3,2352,1,1.0,40.409907,NaN,1.0,10,22.0,1977,0.007122,...,1,264,0,NaN,0,1,B,168106.007630,41.400486,262.02
4,13866,94,2.0,64.285067,38.562517,9.0,16,16.0,1972,0.282798,...,33,8667,2,NaN,0,6,B,343995.102962,38.562517,1579.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1260,61,2.0,49.090728,33.272626,6.0,3,12.0,1981,0.300323,...,52,10311,6,NaN,1,9,B,119367.455796,33.272626,1050.31
9996,16265,27,2.0,64.307684,37.038420,9.0,13,0.0,1977,0.072158,...,2,629,1,NaN,0,0,A,199715.148807,37.038420,275.53
9997,2795,178,1.0,29.648057,16.555363,5.0,3,5.0,1958,0.460556,...,20,4386,14,NaN,1,5,B,165953.912580,16.555363,2049.10
9998,14561,21,1.0,32.330292,22.326870,5.0,3,9.0,1969,0.194489,...,47,8004,3,125.0,3,5,B,171842.411855,22.326870,125.00


### Строим модель

In [19]:
train_df.drop(['Shops_2','Ecology_2','Ecology_3','LifeSquare','Healthcare_1'], axis=1, inplace=True)
test_df.drop(['Shops_2','Ecology_2','Ecology_3'], axis=1, inplace=True)

In [20]:
target_name = 'Price'

In [21]:
X = train_df.drop(columns=target_name)
y = train_df[target_name]

In [22]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=21)

In [23]:
rf_model = RandomForestRegressor(random_state=21, criterion='mse')


In [24]:
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=21)

In [25]:
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_valid)

In [26]:
    print("Train R2:\t" + str(round(r2(y_train, y_train_pred), 3)))
    print("Test R2:\t" + str(round(r2(y_valid, y_test_pred), 3)))

Train R2:	0.959
Test R2:	0.737


### Обработка файла теста

In [27]:
test_df.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,2623.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1146.657263,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1044.744231,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,325.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,1548.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


In [28]:
tmp_df = test_df.copy()
tmp_df['ind'] = [x for x in range(len(tmp_df))]
tmp_target_name = 'LifeSquare'
tmp_df.columns.tolist()
tmp_feature_name = [#'Id',
 #'DistrictId',
 'Rooms',
 'Square',
 #'LifeSquare',
 'KitchenSquare',
 'Floor',
 'HouseFloor',
 'HouseYear',
 'Ecology_1',
 #'Ecology_2',
 #'Ecology_3',
 'Social_1',
 'Social_2',
 'Social_3',
 #'Healthcare_1',
 'Helthcare_2',
 'Shops_1',
 #'Shops_2',
 #'ind'
]
tmp_df_test = tmp_df[tmp_df[tmp_target_name].isna() == True]
tmp_df_train = tmp_df[tmp_df[tmp_target_name].isna() == False]
X_tmp_train = tmp_df_train[tmp_feature_name]
X_tmp_test = tmp_df_test[tmp_feature_name]
y_tmp_train = tmp_df_train[tmp_target_name]
rf_model_tmp = RandomForestRegressor(random_state=8)
rf_model_tmp.fit(X_tmp_train, y_tmp_train)
tmp_df_test[tmp_target_name] = rf_model_tmp.predict(X_tmp_test)
df2 = pd.concat([tmp_df_train, tmp_df_test])
df2 = df2.sort_values(by=['ind'])
test_df[tmp_target_name + '_num'] = df2[tmp_target_name]
test_df

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,LifeSquare_num
0,4567,44,1.0,36.847630,19.094182,5.0,5,9.0,1970,0.036122,24,4378,0,1036.0,1,1,19.094182
1,5925,62,1.0,42.493907,42.568133,10.0,7,17.0,2017,0.072158,2,629,1,NaN,0,0,42.568133
2,960,27,2.0,59.463678,NaN,9.0,19,19.0,1977,0.211401,9,1892,0,NaN,0,1,39.629252
3,3848,23,3.0,49.646030,33.893825,6.0,2,2.0,1965,0.014073,2,475,0,NaN,0,0,33.893825
4,746,74,1.0,53.837056,NaN,1.0,8,17.0,1977,0.309479,35,7715,4,990.0,0,6,34.336919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,10379,29,2.0,43.177521,30.339945,5.0,6,5.0,1962,0.069660,31,6119,4,NaN,1,2,30.339945
4996,16138,38,3.0,93.698122,94.521465,10.0,21,27.0,2018,0.060753,15,2787,2,520.0,0,7,94.521465
4997,3912,101,1.0,33.656723,19.003259,5.0,2,5.0,1966,0.038693,28,6533,1,1015.0,2,5,19.003259
4998,5722,10,1.0,38.635155,20.976257,9.0,8,14.0,1970,0.089040,33,7976,5,NaN,0,11,20.976257


In [29]:
tmp_df = test_df.copy()
tmp_df['ind'] = [x for x in range(len(tmp_df))]
tmp_target_name = 'Healthcare_1'
tmp_df.columns.tolist()
tmp_feature_name = [#'Id',
 #'DistrictId',
 'Rooms',
 'Square',
 #'LifeSquare',
 'KitchenSquare',
 'Floor',
 'HouseFloor',
 'HouseYear',
 'Ecology_1',
 #'Ecology_2',
 #'Ecology_3',
 'Social_1',
 'Social_2',
 'Social_3',
 #'Healthcare_1',
 'Helthcare_2',
 'Shops_1',
 #'Shops_2',
'LifeSquare_num',
 #'ind'
]
tmp_df_test = tmp_df[tmp_df[tmp_target_name].isna() == True]
tmp_df_train = tmp_df[tmp_df[tmp_target_name].isna() == False]
X_tmp_train = tmp_df_train[tmp_feature_name]
X_tmp_test = tmp_df_test[tmp_feature_name]
y_tmp_train = tmp_df_train[tmp_target_name]
rf_model_tmp = RandomForestRegressor(random_state=8)
rf_model_tmp.fit(X_tmp_train, y_tmp_train)
tmp_df_test[tmp_target_name] = rf_model_tmp.predict(X_tmp_test)
df2 = pd.concat([tmp_df_train, tmp_df_test])
df2 = df2.sort_values(by=['ind'])
test_df[tmp_target_name + '_num'] = df2[tmp_target_name]
test_df

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,LifeSquare_num,Healthcare_1_num
0,4567,44,1.0,36.847630,19.094182,5.0,5,9.0,1970,0.036122,24,4378,0,1036.0,1,1,19.094182,1036.00
1,5925,62,1.0,42.493907,42.568133,10.0,7,17.0,2017,0.072158,2,629,1,NaN,0,0,42.568133,457.37
2,960,27,2.0,59.463678,NaN,9.0,19,19.0,1977,0.211401,9,1892,0,NaN,0,1,39.629252,560.92
3,3848,23,3.0,49.646030,33.893825,6.0,2,2.0,1965,0.014073,2,475,0,NaN,0,0,33.893825,162.77
4,746,74,1.0,53.837056,NaN,1.0,8,17.0,1977,0.309479,35,7715,4,990.0,0,6,34.336919,990.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,10379,29,2.0,43.177521,30.339945,5.0,6,5.0,1962,0.069660,31,6119,4,NaN,1,2,30.339945,593.44
4996,16138,38,3.0,93.698122,94.521465,10.0,21,27.0,2018,0.060753,15,2787,2,520.0,0,7,94.521465,520.00
4997,3912,101,1.0,33.656723,19.003259,5.0,2,5.0,1966,0.038693,28,6533,1,1015.0,2,5,19.003259,1015.00
4998,5722,10,1.0,38.635155,20.976257,9.0,8,14.0,1970,0.089040,33,7976,5,NaN,0,11,20.976257,802.24


In [31]:
test_df.drop(['LifeSquare','Healthcare_1'], axis=1, inplace=True)

In [32]:
y_test = rf_model.predict(test_df)

In [33]:
y_test

array([163776.91538478, 114588.6133276 , 146291.5337846 , ...,
       146606.94237997, 188493.60469744, 275054.95710964])

In [34]:
submit = pd.read_csv('sample_submission.csv')

In [35]:
submit['Price'] = y_test
submit.head()

,Id,Price
0,4567,163776.915385
1,5925,114588.613328
2,960,146291.533785
3,3848,129892.459053
4,746,196655.927012


In [36]:
submit.to_csv('rf_submit.csv', index=False)

In [37]:
rf_model = RandomForestRegressor(random_state=21, criterion='mse', n_jobs=-1)
rf_model.fit(X, y)

RandomForestRegressor(n_jobs=-1, random_state=21)

In [38]:
y_test2 = rf_model.predict(test_df)
submit['Price'] = y_test2
submit.to_csv('rf_submit2.csv', index=False)